In [ ]:
// 1. Setting temporal periods
var periods = [
  {label: '2017', year: 2017},
  {label: '2018', year: 2018},
  {label: '2019', year: 2019}
];

// 2. Function to calculate indices and add to each image
function addIndices(image) {
  var ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI');
  var msavi = image.expression(
    '(2 * NIR + 1 - sqrt((2 * NIR + 1) ** 2 - 8 * (NIR - RED))) / 2', {
      'NIR': image.select('B8'),
      'RED': image.select('B4')
    }).rename('MSAVI');

  var ndre = image.normalizedDifference(['B8','B5']).rename('NDRE');
  return image.addBands([ndvi, msavi, ndre]);
}

// 3. Loop on each year and each month from April to July

periods.forEach(function(period) {
  var year = period.year;
  var months = [4, 5 , 6, 7];

  var ndviList = [];
  var ndreList = [];
  var msaviList = [];

  months.forEach(function(month) {
    var startDate = ee.Date.fromYMD(year, month, 1);
    var endDate = startDate.advance(1, 'month');

    var s2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
        .filterDate(startDate, endDate)
        .filterBounds(buffer500)
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
        .map(addIndices);

    var monthlyMean = s2.mean();

    ndviList.push(monthlyMean.select('NDVI').rename('NDVI_' + month));
    ndreList.push(monthlyMean.select('NDRE').rename('NDRE_' + month));
    msaviList.push(monthlyMean.select('MSAVI').rename('MSAVI_' + month));
  });  // <== fermeture correcte de months.forEach()

  // Empilement
  var ndviStack = ee.Image.cat(ndviList);
  var ndreStack = ee.Image.cat(ndreList);
  var msaviStack = ee.Image.cat(msaviList);

  // === Fonction de calcul des stats ===
  function getStats(imgStack, prefix) {
    var bands = imgStack.select([
      prefix + '_4', prefix + '_5', prefix + '_6', prefix + '_7'
    ]);

    var mean = bands.reduce(ee.Reducer.mean()).rename(prefix + '_mean_' + year);
    var std = bands.reduce(ee.Reducer.stdDev()).rename(prefix + '_std_' + year);
    var min = bands.reduce(ee.Reducer.min()).rename(prefix + '_min_' + year);
    var max = bands.reduce(ee.Reducer.max()).rename(prefix + '_max_' + year);
    var amp = max.subtract(min).rename(prefix + '_amplitude_' + year);

    var peakMonth = bands
      .toArray()
      .arrayArgmax()
      .arrayProject([0])
      .arrayFlatten([['index']])
      .add(4) // avril = 4
      .rename(prefix + '_peakMonth_' + year);

    return mean.addBands(std).addBands(amp).addBands(peakMonth);
  }

  // Appliquer pour chaque indice
  var ndviStats = getStats(ndviStack, 'NDVI');
  var ndreStats = getStats(ndreStack, 'NDRE');
  var msaviStats = getStats(msaviStack, 'MSAVI');

  // Combiner tous les résultats
  var finalImage = ndviStats.addBands(ndreStats).addBands(msaviStats);

  // Appliquer les stats par polygone
  var zonalStats = finalImage.reduceRegions({
    collection: buffer500,
    reducer: ee.Reducer.mean(),
    scale: 10
  });

  // Ajouter l’année en attribut
  zonalStats = zonalStats.map(function(f) {
    return f.set('year', year);
  });

  // Afficher dans la console
  print('Zonal stats pour ' + year, zonalStats);
});  // <== fermeture correcte de periods.forEach()

